In [1]:
import time
import re
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

rings_all_link = "https://www.zales.com/rings/rings/c/0106050000?icid=RINGS_VIEWALL" 
driver.get(rings_all_link)


jewels_name_xpath = '//*[contains(concat( " ", @class, " " ), concat( " ", "name", " " ))]//a'
jewels_price_now_xpath = '//*[contains(concat( " ", @class, " " ), concat( " ", "price", " " ))]'
    
SCROLL_PAUSE_TIME = 0.5

only_first_n = 1000 # to avoid blocking of the ip, we only get 1000 objects 
count = 0

last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    jewels_name = driver.find_elements_by_xpath(jewels_name_xpath)
    jewels_price_now = driver.find_elements_by_xpath(jewels_price_now_xpath)
    
    if len(jewels_name)/2 >= only_first_n: #
        break

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

name_list = []
price_now_list = []

name_list = [i.text for i in jewels_name if i.text]
price_now_list = [i.text for i in jewels_price_now if i.text]

pattern_ct = re.compile('((\d+(\/\d+)?)\s+CT\.\s+T\.W\.)|(\d+\.?\d*mm)')
ct_list = []
for i in name_list:
    maybe_match = pattern_ct.search(i)
    if maybe_match is None:
        match = ['Na']
    else:
        match = [maybe_match.group(0)]
    ct_list += match

pattern_price = re.compile('\$[0-9,\.]*') #
cost_list = []
for i in price_now_list:
    maybe_match = pattern_price.search(i)
    if maybe_match is None:
        match = ['Na']
    else:
        match = [maybe_match.group(0)]
    cost_list += match

pattern_discount = re.compile('(?<=\()(.*?)(?<=%)')
discount_list = []
for i in price_now_list:
    maybe_match = pattern_discount.search(i)
    if maybe_match is None:
        match = ['0']
    else:
        match = [maybe_match.group(0)]
    discount_list += match

    
with open('rings.txt', 'w') as f:
    #header 
    header = ['name','size','cost','discount', 'original_name', 'original_price']
    f.write(','.join(header) + '\n')

    #body
    for i in range(len(name_list)):
        f.write(re.sub(pattern_ct, '', name_list[i]).replace(',','').lstrip() + ', ')
        f.write(ct_list[i] + ', ')
        f.write(cost_list[i].replace(',','').lstrip() + ', ') # replace comma with nothing, to avoid misunderstanding in csv files
        f.write(discount_list[i] + ', ')
        f.write(name_list[i].replace(',','').lstrip() + ', ')
        f.write(price_now_list[i].replace(',','').lstrip())
        f.write('\n')
        
        




driver.close()
driver.quit()



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\franc\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


In [11]:
# read
data = pd.read_csv('rings.txt', encoding= 'unicode_escape')
data.head(50)

,name,size,cost,discount,original_name,original_price
0,Composite Diamond Cushion Frame Split Shank Ri...,1 CT. T.W.,$699.98,52%,1 CT. T.W. Composite Diamond Cushion Frame Sp...,$699.98 (52% off)
1,Composite Diamond Split Shank Ring in 10K Whit...,2 CT. T.W.,$999.98,61%,2 CT. T.W. Composite Diamond Split Shank Ring...,$999.98 (61% off)
2,Diamond Vintage-Style Multi-Row Ring in Sterli...,1 CT. T.W.,$399.98,63%,1 CT. T.W. Diamond Vintage-Style Multi-Row Ri...,$399.98 (63% off)
3,Princess-Cut Diamond Double Frame Split Shank ...,1/2 CT. T.W.,$499.00,56%,1/2 CT. T.W. Princess-Cut Diamond Double Fram...,$499.00 (56% off)
4,Diamond Double Pear-Shaped Frame Twist Shank E...,1/2 CT. T.W.,$499.00,56%,1/2 CT. T.W. Diamond Double Pear-Shaped Frame...,$499.00 (56% off)
5,Diamond Past Present Future® Double Oval Frame...,1 CT. T.W.,$899.98,52%,1 CT. T.W. Diamond Past Present Future® Doubl...,$899.98 (52% off)
6,Men's Diamond Triple Row Rectangle-Top Collar...,3/4 CT. T.W.,$999.98,41%,Men's 3/4 CT. T.W. Diamond Triple Row Rectang...,$999.98 (41% off)
7,Men's Square Composite Diamond Double Frame S...,5 CT. T.W.,$3499.98,42%,Men's 5 CT. T.W. Square Composite Diamond Dou...,$3499.98 (42% off)
8,Men's Diamond Ring in 10K Gold,1/10 CT. T.W.,$499.98,34%,Men's 1/10 CT. T.W. Diamond Ring in 10K Gold,$499.98 (34% off)
9,Men's Rectangular Composite Diamond Border St...,7/8 CT. T.W.,$1299.98,35%,Men's 7/8 CT. T.W. Rectangular Composite Diam...,$1299.98 (35% off)


In [55]:
# customized ring

# boilerplate code
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome(ChromeDriverManager().install())

link_customized_ring = 'https://www.zales.com/customize-your-vera-wang-love-engagement-ring/p/V-20298826?icid=CYO_LP:VERA_CONFIG'
#https://www.zales.com/customize-your-vera-wang-love-engagement-ring/p/V-20298826?icid=CYO_LP:VERA_CONFIG
driver.get(link_customized_ring)

# Wait to load page
time.sleep(SCROLL_PAUSE_TIME)

price_xpath = '//*[(@id = "sticky-price")]//span'
restart_xpath = '//*[(@id = "start-over")]'
base_price = driver.find_elements_by_xpath(price_xpath)[1] # i do not like this hard code, pero ni modo
#price_xpath = '//*[(@id = "sticky-price")]//span+//span'

base_price

#pino.text



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\franc\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


InvalidSelectorException: Message: invalid selector: Unable to locate an element with the xpath expression //*[(@id = "sticky-price")]//span+//span because of the following error:
TypeError: Failed to execute 'evaluate' on 'Document': The result is not a node set, and therefore cannot be converted to the desired type.
  (Session info: chrome=94.0.4606.61)
